In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab7").getOrCreate()

24/10/07 10:56:20 WARN Utils: Your hostname, mahe-HP-Z1-G9-Tower-Desktop-PC resolves to a loopback address: 127.0.1.1; using 172.16.57.86 instead (on interface eno1)
24/10/07 10:56:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 10:56:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.option("inferSchema", True).option("header", False).csv("kddcup.data_10_percent_corrected")

In [3]:
column_names = [ "duration", "protocol_type", "service", "flag","src_bytes", "dst_bytes", "land", "wrong_fragment", 
                "urgent","hot", "num_failed_logins", "logged_in", "num_compromised","root_shell", "su_attempted", 
                "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", 
                "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
                "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", 
                "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", 
                "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", 
                "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"]

data = data.toDF(*column_names)
data.head()

24/10/07 10:56:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=181, dst_bytes=5450, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=1, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=8, srv_count=8, serror_rate=0.0, srv_serror_rate=0.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=1.0, diff_srv_rate=0.0, srv_diff_host_rate=0.0, dst_host_count=9, dst_host_srv_count=9, dst_host_same_srv_rate=1.0, dst_host_diff_srv_rate=0.0, dst_host_same_src_port_rate=0.11, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.0, dst_host_srv_serror_rate=0.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0, label='normal.')

In [4]:
from pyspark.sql.functions import col
data.select("label").groupBy("label").count().orderBy(col("count").desc()).show(25)

+----------------+------+
|           label| count|
+----------------+------+
|          smurf.|280790|
|        neptune.|107201|
|         normal.| 97278|
|           back.|  2203|
|          satan.|  1589|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|    warezclient.|  1020|
|       teardrop.|   979|
|            pod.|   264|
|           nmap.|   231|
|   guess_passwd.|    53|
|buffer_overflow.|    30|
|           land.|    21|
|    warezmaster.|    20|
|           imap.|    12|
|        rootkit.|    10|
|     loadmodule.|     9|
|      ftp_write.|     8|
|       multihop.|     7|
|            phf.|     4|
|           perl.|     3|
|            spy.|     2|
+----------------+------+



In [5]:
numeric = data.drop("protocol_type", "service", "flag").cache()
k=23

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler

assembler = VectorAssembler().setInputCols(numeric.columns[:-1]).setOutputCol("featureVector")

scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector") \
.setWithStd(True).setWithMean(False)

kmeans = KMeans().setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster") \
.setFeaturesCol("scaledFeatureVector")

pipeline = Pipeline().setStages([assembler, scaler, kmeans])

model = pipeline.fit(numeric)
predictions = model.transform(numeric)

kmeans_model = model.stages[-1]

24/10/07 10:56:27 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [7]:
training_cost = kmeans_model.summary.trainingCost
print(training_cost)

3346023.3095875485


In [8]:
from pprint import pprint

centers=kmeans_model.clusterCenters()
pprint(centers)

[array([6.52644815e-06, 8.08265265e-03, 3.27392061e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.94561857e-05, 0.00000000e+00,
       2.59328536e-04, 5.76480016e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.89327466e-01,
       4.50757516e-02, 2.62599451e+00, 2.62401682e+00, 4.12270436e-04,
       3.08133412e-04, 1.84094598e-01, 7.72792301e-01, 9.73945053e-04,
       3.93335585e+00, 1.05412738e-01, 1.07043331e-01, 6.07551502e-01,
       2.89510090e-03, 8.09246547e-04, 2.62658852e+00, 2.62454897e+00,
       1.74839170e-04, 2.42750007e-04]),
 array([0.00000000e+00, 9.46618409e-04, 7.17362618e-07, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e

In [9]:
with_cluster = model.transform(numeric)
with_cluster.select("cluster", "label").groupBy("cluster", "label").count().orderBy(col("cluster"), 
                                                                                    col("count").desc()).show(25)

+-------+----------------+------+
|cluster|           label| count|
+-------+----------------+------+
|      0|        neptune.| 86676|
|      0|           nmap.|   102|
|      0|      portsweep.|    23|
|      0|         normal.|    10|
|      0|           imap.|     3|
|      1|          smurf.|280674|
|      1|         normal.|   107|
|      1|          satan.|     1|
|      2|        neptune.| 20457|
|      2|      portsweep.|   746|
|      2|         normal.|   114|
|      2|        ipsweep.|    18|
|      3|         normal.|     2|
|      4|         normal.| 39171|
|      4|    warezclient.|   624|
|      4|           back.|   478|
|      4|           nmap.|   103|
|      4|        ipsweep.|    87|
|      4|            pod.|    74|
|      4|buffer_overflow.|    12|
|      4|           imap.|     8|
|      4|        neptune.|     4|
|      4|       multihop.|     3|
|      4|    warezmaster.|     3|
|      4|     loadmodule.|     3|
+-------+----------------+------+
only showing t

In [10]:
centers_df = spark.createDataFrame([(i, center.tolist()) for i, center in enumerate(centers)], 
                                   ["cluster", "centroid"])

predictions = predictions.join(centers_df, on="cluster")

In [11]:
from pyspark.sql.functions import udf,sqrt
from pyspark.sql.types import FloatType

def calculate_distance(v1, v2):
    return float(sum((v1[i] - v2[i]) ** 2 for i in range(len(v1))))

distance_udf = udf(lambda v1, v2: sum((v1[i] - v2[i]) ** 2 for i in range(len(v1))), FloatType())

predictions = predictions.withColumn("squared_distance", distance_udf(col("scaledFeatureVector"), 
                                                                      col("centroid")))
predictions = predictions.withColumn("distance", sqrt(col("squared_distance")))

In [12]:
from pyspark.sql.functions import mean,stddev

distance_stats = predictions.select(mean("distance").alias("mean_distance"), stddev("distance")
                                    .alias("stddev_distance"))

mean_distance = distance_stats['mean_distance']
stddev_distance = distance_stats['stddev_distance']

anomaly_threshold = distance_stats.mean_distance + 2 * distance_stats.stddev_distance


In [13]:
from pyspark.sql.functions import when

predictions = predictions.withColumn("is_anomaly", when(col("distance") > anomaly_threshold, 1).otherwise(0))

AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_MISSING_FROM_INPUT] Resolved attribute(s) "mean_distance", "stddev_distance" missing from "cluster", "duration", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label", "featureVector", "scaledFeatureVector", "centroid", "squared_distance", "distance" in operator !Project [cluster#5491, duration#101, src_byt

In [ ]:
predictions.select("cluster", "is_anomaly", "distance", "label").show(25)

In [ ]:
spark.stop()